In [ ]:
!pip install hopsworks

In [ ]:
!pip install modal-client

In [14]:
! pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
%env MODAL_TOKEN_ID=ak-1XBshrSs2eXl3c23706ugn 
%env MODAL_TOKEN_SECRET=as-qYqo2kd9V9AVWRb0u6QDST 

env: MODAL_TOKEN_ID=ak-1XBshrSs2eXl3c23706ugn
env: MODAL_TOKEN_SECRET=as-qYqo2kd9V9AVWRb0u6QDST


In [14]:
import modal
import os

from API_call_AQI import get_air_quality_df
from API_call_visual_crossing import get_weather_df

LOCAL = False

if not LOCAL:
    
    stub = modal.Stub()
    image = modal.Image.debian_slim().pip_install(["hopsworks==3.0.4"])


    @stub.function(image=image, schedule=modal.Period(days=1), secrets=[modal.Secret.from_name("HOPSWORKS_API_KEY"),
                                                                        modal.Secret.from_name("AIR_QUALITY_API_KEY"),
                                                                        modal.Secret.from_name("WEATHER_API_KEY")])
    def f():
        g()


def g():
    import hopsworks
    import os
    from datetime import datetime
    project = hopsworks.login()
    fs = project.get_feature_store()

    city = 'Paris'
    date_today = datetime.now().strftime("%Y-%m-%d")

    AIR_QUALITY_API_KEY = os.getenv('AIR_QUALITY_API_KEY')
    WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

    df_air_quality = get_air_quality_df(city, AIR_QUALITY_API_KEY)
    df_weather = get_weather_df(city, date_today, WEATHER_API_KEY)

    air_quality_fg = fs.get_feature_group(
        name='air_quality_fg',
        version=1
    )
    air_quality_fg.insert(df_air_quality)

    weather_fg = fs.get_feature_group(
        name='weather_fg',
        version=1
    )
    weather_fg.insert(df_weather)


if __name__ == "__main__":
    if LOCAL:
        from dotenv import load_dotenv

        load_dotenv()
        g()
    else:
        with stub.run():
            f()


Output()

✓ Initialized. View app at https://modal.com/apps/ap-XjNSwkclERyF4Mf7XWtKhG

Output()

✓ Created objects.
├── 🔨 Created f.
├── 🔨 Mounted /content/API_call_AQI.py at /root/.
└── 🔨 Mounted /content/API_call_visual_crossing.py at /root/.

Output()

...

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching offline feature group backfill job...


Backfill Job started successfully, you can follow the progress at 


https://c.app.hopsworks.ai/p/5315/jobs/named/weather_fg_1_offline_fg_backfill/executions


✓ App completed.